<a href="https://colab.research.google.com/github/aknip/Langchain-etc./blob/main/Langchain_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])

In [ ]:
#!pip install langchain langchainhub langchain_experimental openai streamlit -q
#!pip install google-search-results wikipedia -q

In [ ]:
import json
import os
from getpass import getpass
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string): ··········


In [ ]:
from langchain.agents import Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper, SQLDatabase, WikipediaAPIWrapper
from langchain_experimental.sql import SQLDatabaseChain
from langchain.tools import WikipediaQueryRun
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
import os
import langchain
import openai
import streamlit as st
import textwrap

In [ ]:
# Download "Chinook" Music Sales DB
import os
import requests
import zipfile
fname = 'chinook.zip'
url = 'https://www.sqlitetutorial.net/wp-content/uploads/2018/03/' + fname
r = requests.get(url)
open(fname, 'wb').write(r.content)
zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

In [ ]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential']
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')
search = SerpAPIWrapper(serpapi_api_key=CREDS['SERP-API']['key']['credential'])
db = SQLDatabase.from_uri("sqlite:///chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about real-time events. You should ask targeted questions"
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to answer questions about a big picture or background of something."
    ),
    Tool(
        name="MusicSales",
        func=db_chain.run,
        description="useful for when you need to answer questions about mucis sales in a store. Should be strickly follow the tables info."
    )
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)


# Single prompt

In [ ]:
prompt = "How many people live in the US?"
# prompt = "What is the best selling song in the music store?"
# prompt = "What is the cheapest album in the music store? Tell me the price."
# prompt = "What is the best selling song starting with the letter 'F' in the music store? Tell me title and artist."
# prompt = "How is the weather now in Cologne? Please use Celsius as unit for temperature."
response = agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `Demographics of the United States`


Page: Demographics of the United States
Summary: The United States had an official estimated resident population of 333,287,557 on July 1, 2022, according to the U.S. Census Bureau. This figure includes the 50 states and the District of Columbia but excludes the population of five unincorporated U.S. territories (Puerto Rico, Guam, the U.S. Virgin Islands, American Samoa, and the Northern Mariana Islands) as well as several minor island possessions. The United States is the third most populous country in the world. The Census Bureau showed a population increase of 0.4% for the twelve-month period ending in July 2022, below the world average annual rate of 0.9%. The total fertility rate in the United States estimated for 2022 is 1.665 children per woman, which is below the replacement fertility rate of approximately 2.1.
The U.S. population almost quadrupled during the 20th century 

In [ ]:
print(textwrap.fill(str(response["input"]), 80))
print()
print(textwrap.fill(str(response["output"]), 80))
print()
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
for msg in msg_history:
  print(msg)

How many people live in the US?

As of July 1, 2022, the estimated resident population of the United States is
approximately 333,287,557. This figure includes the 50 states and the District
of Columbia but excludes the population of five unincorporated U.S. territories
and several minor island possessions. The United States is the third most
populous country in the world.

{'role': 'HumanMessage', 'content': 'How is the weather now in Cologne?'}
{'role': 'AIMessage', 'content': 'The current weather in Cologne, Germany is partly cloudy with a temperature of 60 degrees Fahrenheit. The humidity is at 68% and there is a wind speed of 8 mph. The chance of precipitation is 1%.'}
{'role': 'HumanMessage', 'content': 'Please use Celsius as unit for temperature.'}
{'role': 'AIMessage', 'content': 'I apologize for the confusion. The current temperature in Cologne, Germany is approximately 15.5 degrees Celsius. The weather is partly cloudy with a humidity of 68% and wind speed of 8 mph. The chance

# Chatbot with memory

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
messages = [{"role": "assistant", "content": "How can I help you?"}]
for msg in messages:
    #st.chat_message(msg["role"]).write(msg["content"])
    print(msg["role"] + ": " + msg["content"])

assistant: How can I help you?


In [ ]:
prompt = "And the temperature in Celsius?"
messages.append({"role": "user", "content": prompt})
print("user: " + prompt)
response = agent_executor.invoke({"input": prompt})
msg = {"role": "assistant", "content": response["output"]}
messages.append(msg)
print(textwrap.fill("assistant: " + response["output"], 80))
print("\nHistory:")
for msg in messages:
    #st.chat_message(msg["role"]).write(msg["content"])
    print(textwrap.fill(msg["role"] + ": " + msg["content"], 80))

user: And the temperature in Celsius?


> Entering new AgentExecutor chain...

Invoking: `Search` with `current temperature in Celsius`


['National current temperatures map for the United States in degrees celsius.']I'm sorry, but I can only provide a general temperature map for the United States in degrees Celsius. Could you please specify the location you're interested in?

> Finished chain.
assistant: I'm sorry, but I can only provide a general temperature map for the
United States in degrees Celsius. Could you please specify the location you're
interested in?

History:
assistant: How can I help you?
user: And the temperature in Celsius?
assistant: I'm sorry, but I can only provide a general temperature map for the
United States in degrees Celsius. Could you please specify the location you're
interested in?


In [ ]:
print("Memory from response object:")
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
for msg in msg_history:
  print(msg)

Memory from response object:


IndexError: ignored

# Chat

In [ ]:
  !pip install gradio -q
  %load_ext gradio

In [ ]:
def run_gradio_app():
  import gradio as gr
  import random
  import time

  # Theming
  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )
  # Styling: Change max width
  css = """
    .gradio-container {max-width: 800px!important}
    .vspacer1 {margin-top: 20px}
  """

  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# Agent Chat", elem_classes="vspacer1")

      with gr.Tab("Chat"):
        # https://www.gradio.app/docs/chatbot

        chatbot = gr.Chatbot(bubble_full_width=False)
        msg = gr.Textbox()
        clear = gr.ClearButton([msg, chatbot])

        # Init agent and memory
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
        messages = [{"role": "assistant", "content": "How can I help you?"}]
        #chatbot.append(None, "How can I help you?")

        def ask(message, chat_history):
            chat_history.append((message, None))
            messages.append({"role": "user", "content": message})
            return "", chat_history

        def respond(chat_history):
            prompt = chat_history[-1][0] # get prompt from history (last entry)
            response = agent_executor.invoke({"input": prompt})
            msg = {"role": "assistant", "content": response["output"]}
            messages.append(msg)
            chat_history.append((None, response["output"]))
            print("\n\nMemory from response object:")
            print(textwrap.fill(str(response["chat_history"]), 80))
            return chat_history

        msg.submit(ask, [msg, chatbot], [msg, chatbot], queue=False).then(
          respond, chatbot, chatbot
        )


  demo.launch(quiet=True, share=False, debug=True)

run_gradio_app()

<IPython.core.display.Javascript object>



> Entering new AgentExecutor chain...

Invoking: `Search` with `weather in Munich`


{'type': 'weather_result', 'temperature': '65', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '52%', 'wind': '3 mph', 'location': 'Munich, Germany', 'date': 'Friday 1:00 PM', 'weather': 'Sunny'}The weather in Munich, Germany is currently sunny with a temperature of 65 degrees Fahrenheit. The humidity is at 52% and there is a wind speed of 3 mph. There is no precipitation expected.

> Finished chain.


Memory from response object:
[HumanMessage(content='how is the weather in Munich?'), AIMessage(content='The
weather in Munich, Germany is currently sunny with a temperature of 65 degrees
Fahrenheit. The humidity is at 52% and there is a wind speed of 3 mph. There is
no precipitation expected.')]


> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `Munich`


Page: Munich
Summary: Munich ( MEW-nik; German: München [ˈmʏnçn̩] ; Bavarian: Minga [ˈmɪŋ(ː)ɐ] ) is the capital and most 

In [ ]:
# BACKUP - WEG:
def run_gradio_app():
  import gradio as gr
  import random
  import time

  # Theming
  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )
  # Styling: Change max width
  css = """
    .gradio-container {max-width: 800px!important}
    .vspacer1 {margin-top: 50px}
  """

  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# MyApp", elem_classes="vspacer1")
      gr.Markdown("### Optimizing your work with LLMs.")


      #
      # 1. Tab 1
      #
      with gr.Tab("Chat"):
        # https://www.gradio.app/docs/chatbot

        chatbot = gr.Chatbot(bubble_full_width=False)
        msg = gr.Textbox()
        clear = gr.ClearButton([msg, chatbot])


        def respond(message, chat_history):
            bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
            chat_history.append((message, bot_message))
            #time.sleep(2)
            return "", chat_history

        def respond1(message, chat_history):
            chat_history.append((message, None))
            return "", chat_history

        def respond2(chat_history):
            bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
            chat_history.append((None, bot_message))
            chat_history = [['hello', bot_message]] # hack
            time.sleep(2)
            return chat_history

        #msg.submit(respond, [msg, chatbot], [msg, chatbot])


        msg.submit(respond1, [msg, chatbot], [msg, chatbot], queue=False).then(
          respond2, chatbot, chatbot
        )


      #
      # 2. Tab 2
      #
      with gr.Tab("Input Text "):
        gr.Markdown("Please enter text")

        # Input text via UI
        gr.Markdown("### Input your text:")
        text_input = gr.Textbox(label="Enter text", placeholder="Your text here...", lines=10)
        text_output = gr.Textbox(label="Result")


      #
      # 3. Tab 3
      #
      with gr.Tab("Step 2"):
        gr.Markdown("Please select the optimization:")
        radio = gr.Radio(
          ["by headline", "by paragraph", "by §§§"], label="Text split method"
        )
        name = gr.Textbox(label="Name", placeholder="Enter text...")
        output = gr.Textbox(label="Output Box")


  demo.launch(quiet=True, share=False, debug=True)

run_gradio_app()

# Tool Tests

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool, StructuredTool, Tool

In [ ]:
import os
os.environ["LANGCHAIN_TRACING"] = "true"
#llm = OpenAI(temperature=0)
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')

In [ ]:
# Tool 1
def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b

tool = StructuredTool.from_function(multiplier)

agent_executor = initialize_agent(
    [tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
agent_executor.run("What is 3 times 4")

In [ ]:
# Tool 2
def calculate_insurance_premium(insured_sum: int, industry: str) -> float:
    """Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer."""
    premium = insured_sum*0.25
    return premium

tool = StructuredTool.from_function(calculate_insurance_premium, return_direct=True)

agent_executor = initialize_agent(
    [tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
agent_executor.run("Give me the premium for an insured sum of 10000000")



> Entering new AgentExecutor chain...


The user has asked for the premium for an insured sum but has not provided the industry. I need to ask for the industry to be able to calculate the premium. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "Could you please specify the industry for which you want to calculate the insurance premium?"
}
```

> Finished chain.


'Could you please specify the industry for which you want to calculate the insurance premium?'